In [1]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

In [2]:
from src.nlp.preprocessing import DataPreprocessor

INFO : NumExpr defaulting to 8 threads.


In [3]:
import pandas as pd
doc_series = pd.read_csv("temp_data_save.csv")
# doc_series["cleaned_results"] = doc_series.raw_text.apply(lambda sentence: prepare_training_df(sentence))
# doc_series["title"] = doc_series.cleaned_results.apply(lambda d: d[1])
# doc_series["raw_text"] = doc_series.cleaned_results.apply(lambda d: d[0])
# doc_series["cleaned_results"] = doc_series.cleaned_results.apply(lambda d: f"{d[1]} {d[0]}")

In [4]:
data_processor = DataPreprocessor(doc_series.raw_text, n_grams=3)

In [5]:
data_processor.corpus["lemmatized_sentence"] = data_processor.corpus.cleaned.apply(lambda d: data_processor.lemmatize_text(d))

In [6]:
data_processor.corpus["tokens"] = data_processor.corpus.lemmatized_sentence.apply(lambda d: data_processor.generate_ngrams(d))

In [7]:
data_processor.corpus["tokens"] = data_processor.corpus.tokens.apply(lambda d: [" ".join(token) for token in d])

In [8]:
from gensim.corpora import Dictionary
word_dict = Dictionary(data_processor.corpus.tokens.tolist())

INFO : adding document #0 to Dictionary<0 unique tokens: []>
INFO : built Dictionary<3417369 unique tokens: ['able', 'able do', 'able do so', 'able form', 'able form complex']...> from 1740 documents (total 7317807 corpus positions)
INFO : Dictionary lifecycle event {'msg': "built Dictionary<3417369 unique tokens: ['able', 'able do', 'able do so', 'able form', 'able form complex']...> from 1740 documents (total 7317807 corpus positions)", 'datetime': '2022-12-16T12:04:54.379981', 'gensim': '4.2.0', 'python': '3.9.15 (main, Nov 24 2022, 14:39:17) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [9]:
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
word_dict.filter_extremes(no_below=20, no_above=0.5)

INFO : discarding 3405486 tokens: [('able do', 14), ('able do so', 6), ('able form', 7), ('able form complex', 1), ('above', 987), ('above examples', 5), ('above examples suggest', 1), ('above methods', 4), ('above methods used', 1), ('above previous', 2)]...
INFO : keeping 11883 tokens which were in no less than 20 and no more than 870 (=50.0%) documents
INFO : resulting dictionary: Dictionary<11883 unique tokens: ['able', 'adapting', 'addition', 'adjust', 'adjust weights']...>


In [10]:
dictionary_path = "temp_dict_trigrams.txt"

In [11]:
word_dict.save(dictionary_path)

INFO : Dictionary lifecycle event {'fname_or_handle': 'temp_dict_trigrams.txt', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-12-16T12:05:09.736506', 'gensim': '4.2.0', 'python': '3.9.15 (main, Nov 24 2022, 14:39:17) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'saving'}
INFO : saved temp_dict_trigrams.txt


In [12]:
from gensim.models.ldamulticore import LdaMulticore

In [13]:
# Bag-of-words representation of the documents.
data_processor.corpus["embeddings"] = data_processor.corpus.tokens.apply(word_dict.doc2bow)

In [14]:
data_processor.corpus.head()


,raw_text,cleaned,lemmatized_sentence,tokens,embeddings
0,387 \nNeural Net and Traditional Classifiers ...,387 Neural Net and Traditional Classifiers Wil...,neural net traditional classifiers abstract pr...,"[neural, net, traditional, classifiers, abstra...","[(0, 2), (1, 1), (2, 2), (3, 1), (4, 1), (5, 3..."
1,1 \nCONNECTIVITY VERSUS ENTROPY \nYaser S. Abu...,1 CONNECTIVITY VERSUS ENTROPY Yaser S Abu Most...,connectivity entropy abstract connectivity neu...,"[connectivity, entropy, abstract, connectivity...","[(12, 1), (14, 1), (15, 1), (24, 1), (30, 1), ..."
2,9 \nStochastic Learning Networks and their Ele...,9 Stochastic Learning Networks and their Elect...,stochastic learning networks electronic implem...,"[stochastic, learning, networks, electronic, i...","[(2, 4), (3, 2), (12, 1), (15, 1), (17, 3), (2..."
3,22 \nLEARNING ON A GENERAL NETWORK \nAmir F. A...,22 LEARNING ON A GENERAL NETWORK Amir F Atiya ...,learning general paper generalizes backpropaga...,"[learning, general, paper, generalizes, backpr...","[(3, 1), (13, 1), (20, 1), (24, 1), (27, 1), (..."
4,31 \nAN ARTIFICIAL NEURAL NETWORK FOR SPATIO- ...,31 AN ARTIFICIAL NEURAL NETWORK FOR SPATIO TEM...,artificial neural network spatio temporal bipo...,"[artificial, neural, network, spatio, temporal...","[(2, 1), (11, 2), (19, 2), (20, 2), (22, 2), (..."


In [16]:
word_dict[0]
baseline_lda_model = LdaMulticore(
    corpus = data_processor.corpus["embeddings"].tolist(),
    id2word=word_dict.id2token,
    workers=4,
    alpha=0.25,
    num_topics=5,
    minimum_probability=0.2,
    passes=20,
    per_word_topics=True
)

INFO : using symmetric eta at 0.2
INFO : using serial LDA version on this node
INFO : running online LDA training, 5 topics, 20 passes over the supplied corpus of 1740 documents, updating every 8000 documents, evaluating every ~1740 documents, iterating 50x with a convergence threshold of 0.001000
INFO : training LDA model using 4 processes
INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #1740/1740, outstanding queue size 1
INFO : topic #0 (0.250): 0.005*"state" + 0.004*"units" + 0.003*"hidden" + 0.003*"layer" + 0.003*"weight" + 0.002*"noise" + 0.002*"neurons" + 0.002*"unit" + 0.002*"matrix" + 0.002*"distribution"
INFO : topic #1 (0.250): 0.003*"units" + 0.003*"neurons" + 0.002*"image" + 0.002*"unit" + 0.002*"distribution" + 0.002*"layer" + 0.002*"state" + 0.002*"hidden" + 0.002*"recognition" + 0.002*"weight"
INFO : topic #2 (0.250): 0.004*"units" + 0.003*"state" + 0.003*"distribution" + 0.003*"neuron" + 0.003*"neurons" + 0.003*"layer" + 0.002*"noise" + 0.002*"recognitio

INFO : topic #2 (0.250): 0.008*"neurons" + 0.007*"neuron" + 0.006*"cells" + 0.006*"cell" + 0.004*"synaptic" + 0.004*"response" + 0.004*"activity" + 0.003*"circuit" + 0.003*"signal" + 0.003*"firing"
INFO : topic #3 (0.250): 0.006*"distribution" + 0.005*"gaussian" + 0.004*"matrix" + 0.004*"mixture" + 0.004*"noise" + 0.004*"likelihood" + 0.003*"density" + 0.003*"basis" + 0.003*"hidden" + 0.003*"dimensional"
INFO : topic #4 (0.250): 0.008*"units" + 0.007*"hidden" + 0.006*"recognition" + 0.006*"layer" + 0.005*"speech" + 0.004*"unit" + 0.004*"trained" + 0.004*"word" + 0.003*"net" + 0.003*"state"
INFO : topic diff=0.103323, rho=0.356462
INFO : -8.285 per-word bound, 311.8 perplexity estimate based on a held-out corpus of 1740 documents with 2015059 words
INFO : PROGRESS: pass 7, dispatched chunk #0 = documents up to #1740/1740, outstanding queue size 1
INFO : topic #0 (0.250): 0.010*"state" + 0.004*"optimal" + 0.004*"weight" + 0.003*"algorithms" + 0.003*"gradient" + 0.003*"control" + 0.003*"s

INFO : topic diff=0.069310, rho=0.268511
INFO : -8.259 per-word bound, 306.3 perplexity estimate based on a held-out corpus of 1740 documents with 2015059 words
INFO : PROGRESS: pass 13, dispatched chunk #0 = documents up to #1740/1740, outstanding queue size 1
INFO : topic #0 (0.250): 0.010*"state" + 0.005*"optimal" + 0.004*"weight" + 0.004*"control" + 0.004*"algorithms" + 0.003*"gradient" + 0.003*"states" + 0.003*"convergence" + 0.003*"let" + 0.003*"generalization"
INFO : topic #1 (0.250): 0.012*"image" + 0.008*"images" + 0.006*"object" + 0.006*"visual" + 0.005*"feature" + 0.005*"recognition" + 0.005*"features" + 0.004*"objects" + 0.004*"representation" + 0.004*"distance"
INFO : topic #2 (0.250): 0.009*"neurons" + 0.008*"neuron" + 0.007*"cells" + 0.006*"cell" + 0.005*"response" + 0.005*"synaptic" + 0.005*"activity" + 0.004*"signal" + 0.004*"circuit" + 0.004*"frequency"
INFO : topic #3 (0.250): 0.007*"distribution" + 0.006*"gaussian" + 0.005*"matrix" + 0.004*"likelihood" + 0.004*"mixt

INFO : topic #2 (0.250): 0.009*"neurons" + 0.008*"neuron" + 0.007*"cells" + 0.006*"cell" + 0.005*"response" + 0.005*"synaptic" + 0.005*"activity" + 0.004*"signal" + 0.004*"circuit" + 0.004*"frequency"
INFO : topic #3 (0.250): 0.007*"distribution" + 0.006*"gaussian" + 0.005*"matrix" + 0.004*"noise" + 0.004*"likelihood" + 0.004*"mixture" + 0.004*"density" + 0.004*"variables" + 0.003*"log" + 0.003*"variance"
INFO : topic #4 (0.250): 0.011*"units" + 0.009*"hidden" + 0.007*"layer" + 0.006*"unit" + 0.006*"recognition" + 0.005*"speech" + 0.005*"trained" + 0.004*"net" + 0.004*"patterns" + 0.004*"word"
INFO : topic diff=0.044130, rho=0.218896
INFO : -8.249 per-word bound, 304.3 perplexity estimate based on a held-out corpus of 1740 documents with 2015059 words
INFO : LdaMulticore lifecycle event {'msg': 'trained LdaMulticore<num_terms=11883, num_topics=5, decay=0.5, chunksize=2000> in 622.83s', 'datetime': '2022-12-16T12:30:12.731954', 'gensim': '4.2.0', 'python': '3.9.15 (main, Nov 24 2022, 14

In [17]:
baseline_lda_model.print_topics()

INFO : topic #0 (0.250): 0.011*"state" + 0.005*"optimal" + 0.004*"weight" + 0.004*"control" + 0.004*"algorithms" + 0.003*"states" + 0.003*"gradient" + 0.003*"convergence" + 0.003*"let" + 0.003*"generalization"
INFO : topic #1 (0.250): 0.013*"image" + 0.009*"images" + 0.007*"object" + 0.006*"visual" + 0.005*"feature" + 0.005*"recognition" + 0.005*"features" + 0.004*"objects" + 0.004*"position" + 0.004*"representation"
INFO : topic #2 (0.250): 0.009*"neurons" + 0.008*"neuron" + 0.007*"cells" + 0.006*"cell" + 0.005*"response" + 0.005*"synaptic" + 0.005*"activity" + 0.004*"signal" + 0.004*"circuit" + 0.004*"frequency"
INFO : topic #3 (0.250): 0.007*"distribution" + 0.006*"gaussian" + 0.005*"matrix" + 0.004*"noise" + 0.004*"likelihood" + 0.004*"mixture" + 0.004*"density" + 0.004*"variables" + 0.003*"log" + 0.003*"variance"
INFO : topic #4 (0.250): 0.011*"units" + 0.009*"hidden" + 0.007*"layer" + 0.006*"unit" + 0.006*"recognition" + 0.005*"speech" + 0.005*"trained" + 0.004*"net" + 0.004*"pat

[(0,
  '0.011*"state" + 0.005*"optimal" + 0.004*"weight" + 0.004*"control" + 0.004*"algorithms" + 0.003*"states" + 0.003*"gradient" + 0.003*"convergence" + 0.003*"let" + 0.003*"generalization"'),
 (1,
  '0.013*"image" + 0.009*"images" + 0.007*"object" + 0.006*"visual" + 0.005*"feature" + 0.005*"recognition" + 0.005*"features" + 0.004*"objects" + 0.004*"position" + 0.004*"representation"'),
 (2,
  '0.009*"neurons" + 0.008*"neuron" + 0.007*"cells" + 0.006*"cell" + 0.005*"response" + 0.005*"synaptic" + 0.005*"activity" + 0.004*"signal" + 0.004*"circuit" + 0.004*"frequency"'),
 (3,
  '0.007*"distribution" + 0.006*"gaussian" + 0.005*"matrix" + 0.004*"noise" + 0.004*"likelihood" + 0.004*"mixture" + 0.004*"density" + 0.004*"variables" + 0.003*"log" + 0.003*"variance"'),
 (4,
  '0.011*"units" + 0.009*"hidden" + 0.007*"layer" + 0.006*"unit" + 0.006*"recognition" + 0.005*"speech" + 0.005*"trained" + 0.004*"net" + 0.004*"patterns" + 0.004*"word"')]

In [18]:
from gensim.models import CoherenceModel

In [20]:
coherence_model_lda = CoherenceModel(model=baseline_lda_model, texts=data_processor.corpus.tokens.tolist(), dictionary=word_dict, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

INFO : using ParallelWordOccurrenceAccumulator<processes=7, batch_size=64> to estimate probabilities from sliding windows
INFO : 1 batches submitted to accumulate stats from 64 documents (309434 virtual)
INFO : 2 batches submitted to accumulate stats from 128 documents (600331 virtual)
INFO : 3 batches submitted to accumulate stats from 192 documents (841782 virtual)
INFO : 4 batches submitted to accumulate stats from 256 documents (1103909 virtual)
INFO : 5 batches submitted to accumulate stats from 320 documents (1327582 virtual)
INFO : 6 batches submitted to accumulate stats from 384 documents (1568367 virtual)
INFO : 7 batches submitted to accumulate stats from 448 documents (1802147 virtual)
INFO : 8 batches submitted to accumulate stats from 512 documents (2065111 virtual)
INFO : 9 batches submitted to accumulate stats from 576 documents (2310387 virtual)
INFO : 10 batches submitted to accumulate stats from 640 documents (2574365 virtual)
INFO : 11 batches submitted to accumulate


Coherence Score:  0.8198481147665981


In [25]:
import numpy as np

In [28]:
def get_vec_lda(model, corpus, k):
    """
    Get the LDA vector representation (probabilistic topic assignments for all documents)
    :return: vec_lda with dimension: (n_doc * n_topic)
    """
    n_doc = len(corpus)
    vec_lda = np.zeros((n_doc, k))
    for i in range(n_doc):
        # get the distribution for the i-th document in corpus
        for topic, prob in model.get_document_topics(corpus[i]):
            vec_lda[i, topic] = prob
    return vec_lda

In [29]:
vec = get_vec_lda(baseline_lda_model, data_processor.corpus.embeddings.tolist(), 5)

In [31]:
vec.shape

(1740, 5)

In [33]:
from sentence_transformers import SentenceTransformer

In [34]:
model = SentenceTransformer('bert-base-nli-max-tokens')
vec = np.array(model.encode(sentences, show_progress_bar=True))

INFO : Load pretrained SentenceTransformer: bert-base-nli-max-tokens


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/397 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

INFO : Use pytorch device: cuda


NameError: name 'sentences' is not defined